---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
#%matplotlib notebook
def graph_identification():
 #   fig, axs = plt.subplots(3, 2, sharex = True, sharey = True)
 #   x = 0
#    y = 0
    # Your Code Here
 #   for graph in P1_Graphs:
  #      in_degrees = graph.degree()
    #    in_degree_values = sorted(set(in_degrees.values()))
   #     histogram = [list(in_degrees.values()).count(i)/float(nx.number_of_nodes(graph)) for i in in_degree_values]
  #      axs[x, y].plot(in_degree_values,histogram, 'o')
  #      if y < 1:
  #          y+= 1
 #       elif y == 1:
#            y = 0
 #           x += 1
 #   print(type(axs))
 #   plt.show()
 #   plt.setp(axs, xscale='log', yscale = 'log')
 #   print(nx.average_clustering(P1_Graphs[1]))
#    print(nx.average_clustering(P1_Graphs[2]))
 #   print(nx.average_clustering(P1_Graphs[4]))
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']
#graph_identification()

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    from sklearn.metrics import roc_curve, auc, roc_auc_score
    
    # Your Code Here
    ## preparing data
    degCent = nx.degree_centrality(G)
    closeCent = nx.closeness_centrality(G)
    btwnCent = nx.betweenness_centrality(G, normalized = True, endpoints = False)
    hits = nx.hits(G)
    hubs = hits[0]
    authority = hits[1]
    test = G.nodes(data = True)
    df = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
    df['Degree'] = degCent.values()
    df['Closeness'] = closeCent.values()
    df['Betweenness'] = btwnCent.values()
    df['Hub'] = hubs.values()
    df['Authority'] = authority.values()
    test = df[df["ManagementSalary"].isnull()]
    train = df[df["ManagementSalary"].isnull() == False]

    ##training sk classifier

    features = ["Degree", "Closeness", "Betweenness", "Hub", "Authority"]
    X = train[features]
    y = train["ManagementSalary"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    svm = SVC(kernel = 'rbf', C = 1, gamma = 0.001, probability = True).fit(X_train, y_train)
    predict = svm.predict_proba(X_test)
  #  y_score_lr = svm.decision_function(X_test)
   # fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
   # roc_auc_lr = auc(fpr_lr, tpr_lr)
    predict_test = svm.predict_proba(test[features])
    probs = predict_test[:,1]
    test["Probabilities"] = probs
    ans = test["Probabilities"]

    #grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100], 'C': [1, 10 ,100]}

    #grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
    #grid_clf_acc.fit(X_train, y_train)
    #y_decision_fn_scores_acc = grid_clf_acc.decision_function(X_test) 
    #
    #print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
    #print('Grid best score (accuracy): ', grid_clf_acc.best_score_)

    # alternative metric to optimize over grid parameters: AUC
    #grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
    #grid_clf_auc.fit(X_train, y_train)
    #y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test) 

    #print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
    #print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
    #print('Grid best score (AUC): ', grid_clf_auc.best_score_)

    return ans
salary_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1       0.819492
2       0.959026
5       0.996591
8       0.733563
14      0.923422
18      0.871612
27      0.870308
30      0.901497
31      0.801947
34      0.601282
37      0.658286
40      0.831573
45      0.799297
54      0.806991
55      0.789696
60      0.789960
62      0.999998
65      0.955951
77      0.638324
79      0.814530
97      0.567582
101     0.640619
103     0.839441
108     0.791135
113     0.948335
122     0.545633
141     0.933799
142     0.991429
144     0.506985
145     0.923788
          ...   
913     0.481414
914     0.522265
915     0.149933
918     0.595537
923     0.207763
926     0.428768
931     0.460878
934     0.287726
939     0.234268
944     0.170024
945     0.245896
947     0.317864
950     0.832090
951     0.294884
953     0.440375
959     0.212301
962     0.168168
963     0.778814
968     0.221532
969     0.218172
974     0.353485
984     0.266793
987     0.390344
989     0.228882
991     0.240636
992     0.267458
994     0.223875
996     0.1760

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [7]:
def new_connections_predictions():
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    from sklearn.metrics import roc_curve, auc, roc_auc_score
    
    # Your Code Here
    jaccard = list(nx.jaccard_coefficient(G, ebunch = future_connections.index))
    ra = list(nx.resource_allocation_index(G, ebunch = future_connections.index))
    pa = list(nx.preferential_attachment(G, ebunch = future_connections.index))
    future_connections["Jaccard"] = [z for (x, y, z) in jaccard]
    future_connections["Preferential Attachment"] = [z for (x , y, z) in pa]
    future_connections["Resource Allocation"] = [z for (x, y, z) in ra]
    test = future_connections[future_connections["Future Connection"].isnull()]
    train = future_connections[future_connections["Future Connection"].isnull() == False]
    
    ## training sk classifier

    features = ["Jaccard", "Preferential Attachment", "Resource Allocation"]
    X = train[features]
    y = train["Future Connection"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    lr = LogisticRegression(C = 1).fit(X_train, y_train)
    predict = lr.predict_proba(X_test)
    y_score_lr = lr.decision_function(X_test)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_lr)
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    predict_test = lr.predict_proba(test[features])
    test["Probabilities"] = predict_test[:,1]
    ans = test["Probabilities"]

    #grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10, 100], 'C': [1, 10 ,100]}

    #grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
    #grid_clf_acc.fit(X_train, y_train)
    #y_decision_fn_scores_acc = grid_clf_acc.decision_function(X_test) 
    #
    #print('Grid best parameter (max. accuracy): ', grid_clf_acc.best_params_)
    #print('Grid best score (accuracy): ', grid_clf_acc.best_score_)

    # alternative metric to optimize over grid parameters: AUC
    #grid_clf_auc = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc')
    #grid_clf_auc.fit(X_train, y_train)
    #y_decision_fn_scores_auc = grid_clf_auc.decision_function(X_test) 

    #print('Test set AUC: ', roc_auc_score(y_test, y_decision_fn_scores_auc))
    #print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
    #print('Grid best score (AUC): ', grid_clf_auc.best_score_)

    return ans
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.028000
(542, 751)    0.012723
(20, 426)     0.425316
(50, 989)     0.012597
(942, 986)    0.012465
(324, 857)    0.012615
(13, 710)     0.272099
(19, 271)     0.168912
(319, 878)    0.012555
(659, 707)    0.012710
(49, 843)     0.012516
(208, 893)    0.012679
(377, 469)    0.019521
(405, 999)    0.019596
(129, 740)    0.017132
(292, 618)    0.054311
(239, 689)    0.012546
(359, 373)    0.016257
(53, 523)     0.043212
(276, 984)    0.012522
(202, 997)    0.012497
(604, 619)    0.300250
(270, 911)    0.012548
(261, 481)    0.074424
(200, 450)    0.901199
(213, 634)    0.012762
(644, 735)    0.165335
(346, 553)    0.013084
(521, 738)    0.013739
(422, 953)    0.025134
                ...   
(672, 848)    0.012548
(28, 127)     0.742387
(202, 661)    0.013140
(54, 195)     0.999984
(295, 864)    0.012634
(814, 936)    0.012742
(839, 874)    0.012465
(139, 843)    0.012623
(461, 544)    0.014225
(68, 487)     0.014313
(622, 932)    0.012664
(504, 936)    0.022647
(479, 528) 